In [32]:
import os
from copy import deepcopy
import random
import time
import math
import gc
from multiprocessing import Process, Manager
from tqdm import tqdm  
import pandas as pd

%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [35]:
%%cython

cimport cython
from copy import deepcopy

cdef class Connect4:
    cdef public long long bitboards[2]
    cdef public int heights[7]
    cdef public int counter

    def __init__(self):
        self.bitboards[0] = 0
        self.bitboards[1] = 0
        self.heights[:] = [0, 7, 14, 21, 28, 35, 42]
        self.counter = 0
    
    def reset(self, Connect4 connect4):
        self.bitboards[0] = connect4.bitboards[0]
        self.bitboards[1] = connect4.bitboards[1]
        self.counter = connect4.counter
        for i in range(len(self.heights)):
            self.heights[i] = connect4.heights[i]

    def printState(self):
        print("===========================")
        for row in reversed(range(6)):
            for col in range(7):
                bit_index = col * 7 + row
                bit_mask = 1 << bit_index

                if self.bitboards[0] & bit_mask:
                    cell = f"\033[94mO\033[0m"
                elif self.bitboards[1] & bit_mask:
                    cell = f"\033[91mX\033[0m"
                else:
                    cell = " "

                if col < 6:
                    print(f" {cell} |", end="")
                else:
                    print(f" {cell} ")
        print("===========================")
        print(" 1 | 2 | 3 | 4 | 5 | 6 | 7 ")


    def updateGameState(self, int move):
        cdef long long move1
        if move not in self.checkAvailableMoves():
            raise ValueError("Invalid move")
        move1 = <long long>1 << self.heights[move - 1]
        self.bitboards[self.counter & 1] ^= move1
        self.heights[move - 1] += 1
        self.counter += 1

    def checkPlayerWon(self, str player):
        cdef long long bitboard
        cdef long long bb
        cdef int d
        if player == "O":
            bitboard = self.bitboards[0]
        else:
            bitboard = self.bitboards[1]

        for d in [1, 7, 6, 8]:
            bb = bitboard & (bitboard >> d)
            if bb & (bb >> (2 * d)):
                return True
        return False

    def checkTie(self):
        return not self.checkAvailableMoves() and not self.checkPlayerWon("O") and not self.checkPlayerWon("X")

    def checkGameResult(self):
        if self.checkPlayerWon("O"):
            return "O"
        elif self.checkTie():
            return "-"
        else:
            return "X"

    def checkGameOver(self):
        cdef str last_player
        last_player = "O" if self.current_player() == 1 else "X"
        return self.checkPlayerWon(last_player) or self.checkTie()

    def checkAvailableMoves(self):
        cdef int col
        cdef long long TOP = 0b1000000100000010000001000000100000010000001000000
        valid_moves = []
        for col in range(7):
            if (TOP & (<long long>1 << self.heights[col])) == 0:
                valid_moves.append(col + 1)
        return valid_moves

    def current_player(self):
        return self.counter & 1

    def flatten_board(self, board: list[list[str]]) -> list[str]:
        return [cell for row in board for cell in row]

    def bitboard_to_matrix(self) -> list[list[str]]:
        cdef long long bitboardO = self.bitboards[0]
        cdef long long bitboardX = self.bitboards[1]
        cdef list matrix = [["-" for _ in range(7)] for _ in range(6)]
        cdef int col, row
        cdef long long bit_index, mask

        for col in range(7):
            for row in range(6):
                bit_index = col * 7 + row
                mask = <long long>1 << bit_index

                if bitboardO & mask:
                    matrix[5 - row][col] = "O"
                elif bitboardX & mask:
                    matrix[5 - row][col] = "X"

        return matrix

    def matrix_to_bitboard(self, matrix: list) -> tuple:
        cdef long long bitboardO = 0
        cdef long long bitboardX = 0
        cdef int col, row, counter = 0
        cdef long long bit_index
        cdef int heights[7]
        heights[:] = [0, 7, 14, 21, 28, 35, 42]
    
        # Fill bitboards and compute heights
        for col in range(7):
            for row in range(6):
                cell = matrix[5 - row][col]  # Reverse row mapping to match bitboard_to_matrix
                bit_index = col * 7 + row
                if cell == "O":
                    bitboardO |= <long long>1 << bit_index
                    counter += 1
                    heights[col] += 1
                elif cell == "X":
                    bitboardX |= <long long>1 << bit_index
                    counter += 1
                    heights[col] += 1

        self.bitboards[0] = bitboardO
        self.bitboards[1] = bitboardX
        self.counter = counter
        for i in range(len(self.heights)):
            self.heights[i] = heights[i]

Content of stdout:
_cython_magic_9c9892d50e22ea4e1dc1a971bcf70e5e95e0bb912fa1ba47de3dcf373b472e7a.c
C:\Users\arcan\.ipython\cython\_cython_magic_9c9892d50e22ea4e1dc1a971bcf70e5e95e0bb912fa1ba47de3dcf373b472e7a.c(10437): warning C4244: '=': conversion from 'Py_ssize_t' to 'long', possible loss of data
   Creating library C:\Users\arcan\.ipython\cython\Users\arcan\.ipython\cython\_cython_magic_9c9892d50e22ea4e1dc1a971bcf70e5e95e0bb912fa1ba47de3dcf373b472e7a.cp312-win_amd64.lib and object C:\Users\arcan\.ipython\cython\Users\arcan\.ipython\cython\_cython_magic_9c9892d50e22ea4e1dc1a971bcf70e5e95e0bb912fa1ba47de3dcf373b472e7a.cp312-win_amd64.exp
Generating code
Finished generating code

In [25]:
import math
from copy import deepcopy
import random
import gc
import time

class Node:
    def __init__(self, parent: 'Node', move: int, depth: int):
        self.Q = 0
        self.N = 0
        self.depth = depth
        if self.depth % 2 == 1:
            self.playerTurn = "O"
        else:
            self.playerTurn = "X"
        self.move = move
        self.parent = parent
        self.children = {}

class MctsAlgo:
    def __init__(self, C: float = math.sqrt(2), reset: bool = True, connect4: Connect4 = Connect4(), drawValue: float = 0):
        self.C = C
        self.root = Node(None, None, 0)
        self.connect4 = connect4
        self.currentState = self.root
        self.iteration = 0
        self.actualState = self.root
        self.resetTree = reset
        self.drawValue = drawValue
        self.runTimes = []

    def reset(self, connect4Actual: Connect4):
        self.connect4.reset(connect4Actual)
        self.currentState = self.actualState

    def updateMCTSState(self, move: int):
        self.currentState = self.currentState.children[move]

    def updateAfterAdversaryTurn(self, connect4Actual: Connect4, moveBefore: int):
        if len(self.actualState.children) == 0: 
            self.reset(connect4Actual)
            self.expansion_phase(checkIfGameFinished=False)          
        self.actualState = self.actualState.children[moveBefore]

    def selection_phase(self) -> None:
        self.currentState = self.actualState

        while self.currentState.children:
            children = self.currentState.children
            unexplored = [move for move, node in children.items() if node.N == 0]
            if unexplored:
                randomChoice = random.choice(unexplored)
                self.updateMCTSState(randomChoice)
                self.connect4.updateGameState(randomChoice)
                return

            # Compute UCB values for all children
            ucb_values = {move: (node.Q / node.N) + (self.C * math.sqrt((math.log(node.parent.N) / node.N))) for move, node in children.items()}
            max_ucb = max(ucb_values.values())
            # In case of ties, randomly choose among the best
            best_moves = [move for move, val in ucb_values.items() if val == max_ucb]
            randomChoice = random.choice(best_moves)
            self.updateMCTSState(randomChoice)
            self.connect4.updateGameState(randomChoice)

    def expansion_phase(self, checkIfGameFinished: bool) -> bool:
        if checkIfGameFinished:
            isGameFinished = self.connect4.checkGameOver()
            if isGameFinished:
                return False
        availableMoves = self.connect4.checkAvailableMoves()
        for move in availableMoves:
            self.currentState.children[move] = Node(self.currentState, move, self.currentState.depth + 1)
        return True

    def simulation_phase(self, wasExpansionSuccessful: bool) -> str:
        if wasExpansionSuccessful == False:
            gameResult = self.connect4.checkGameResult()
            return gameResult
        moves = [key for key in self.currentState.children.keys()]
        randomChoice = random.choice(moves)
        self.updateMCTSState(randomChoice)
        self.connect4.updateGameState(randomChoice)
        while not self.connect4.checkGameOver():
            self.connect4.updateGameState(random.choice(self.connect4.checkAvailableMoves()))
        gameResult = self.connect4.checkGameResult()
        return gameResult
        
    def backPropagation_phase(self, gameResult: str):
        while self.currentState != self.actualState:
            self.currentState.N += 1
            if self.currentState.playerTurn == gameResult:
                self.currentState.Q += 1
            elif gameResult == "-":
                self.currentState.Q += self.drawValue
            self.currentState = self.currentState.parent
        self.currentState.N += 1
        if self.currentState.playerTurn == gameResult:
            self.currentState.Q += 1
        elif gameResult == "-":
            self.currentState.Q += self.drawValue
        self.iteration += 1

    def run_mcts(self, iterations: int, connect4Actual: Connect4, dataset = None):
        start_time = time.time()  # Start timing
        if self.resetTree:
            self.actualState.children = {}   # Resets tree by clearing every child node
            self.currentState.children = {}  # Definetely resets tree by clearing every child node
            gc.collect()
        if len(self.actualState.children) == 0: 
            self.reset(connect4Actual)
            self.expansion_phase(checkIfGameFinished=False)
        if dataset is None:
            for i in range(iterations):
                self.reset(connect4Actual)
                self.selection_phase()
                wasExpansionSuccessful = self.expansion_phase(checkIfGameFinished=True)
                gameResult = self.simulation_phase(wasExpansionSuccessful)
                self.backPropagation_phase(gameResult)
        else:
            matrix = self.connect4.bitboard_to_matrix()
            flat_board = self.connect4.flatten_board(matrix)
            for i in range(iterations):
                if i == iterations//5:
                    bestMove = self.choose_best_move(datasetFlag=True)
                    flat_board.append(str(bestMove))
                elif i == iterations//5*2:
                    bestMove = self.choose_best_move(datasetFlag=True)
                    flat_board.append(str(bestMove))
                elif i == iterations//5*3:
                    bestMove = self.choose_best_move(datasetFlag=True)
                    flat_board.append(str(bestMove))
                elif i == iterations//5*4:
                    bestMove = self.choose_best_move(datasetFlag=True)
                    flat_board.append(str(bestMove))
                elif i == iterations//5*5-1:
                    bestMove = self.choose_best_move(datasetFlag=True)
                    flat_board.append(str(bestMove))
    
                self.reset(connect4Actual)
                self.selection_phase()
                wasExpansionSuccessful = self.expansion_phase(checkIfGameFinished=True)
                gameResult = self.simulation_phase(wasExpansionSuccessful)
                self.backPropagation_phase(gameResult)
            dataset.append(flat_board)
        end_time = time.time()  # End timing
        self.runTimes.append(round(end_time - start_time, 4))

    def choose_best_move(self, showStats: bool = False, datasetFlag: bool = False, testSoftMax: bool = False, temperature: float = 0.5) -> int:
        def stable_softmax(visits, temperature):
            if temperature == 0:
                max_index = visits.index(max(visits))
                return [1 if i == max_index else 0 for i in range(len(visits))]

            # Apply log for better smoothing and scaling
            scaled = [math.log(v + 1) / temperature for v in visits]
            max_scaled = max(scaled)
            exp_values = [math.exp(s - max_scaled) for s in scaled]
            total = sum(exp_values)
            return [ev / total for ev in exp_values]

        if testSoftMax:
            # Use softmax sampling
            moves = list(self.actualState.children.keys())
            visit_counts = [self.actualState.children[move].N for move in moves]
            probabilities = stable_softmax(visit_counts, temperature)
            bestMove = random.choices(moves, weights=probabilities, k=1)[0]
            if showStats:
                self.print_childrenStats(bestMove, probabilities)
            self.actualState = self.actualState.children[bestMove]
            self.actualState.parent = None
            self.currentState.parent = None
            gc.collect()
        elif datasetFlag:
            # Use softmax sampling
            bestMove = max(self.actualState.children.items(), key=lambda item: item[1].N)[0]
        else:
            # Choose the move with highest visit count
            bestMove = max(self.actualState.children.items(), key=lambda item: item[1].N)[0]
            if showStats:
                self.print_childrenStats(bestMove)
            # Update internal state
            self.actualState = self.actualState.children[bestMove]
            self.actualState.parent = None
            self.currentState.parent = None
            gc.collect()

        return bestMove

    def print_childrenStats(self, bestMove: int, probabilities: list[int] = None):
        for move, child in self.actualState.children.items():
            if move == bestMove:
                if probabilities is None:
                    print(f"\033[93m{move}: {child.Q:>7} / {child.N:<7}\033[0m")
                else:
                    print(f"\033[93m{move}: {child.Q:>7} / {child.N:<7}\t{round(probabilities[list(self.actualState.children).index(move)], 2)}%\033[0m")
            else:
                if probabilities is None:
                    print(f"{move}: {child.Q:>7} / {child.N:<7}")
                else:
                    print(f"{move}: {child.Q:>7} / {child.N:<7}\t{round(probabilities[list(self.actualState.children).index(move)], 2)}%")


In [ ]:
import math
import gc
from multiprocessing import Process, Manager
from tqdm import tqdm  
import os 
import pandas as pd

def user_first_vs_AI(c_constant_mcts: float, iterations: int, reset: bool, drawValue: float, showMCTSTime: bool, showNodesStats: bool):
    mcts = MctsAlgo(C=c_constant_mcts, reset=reset, drawValue=drawValue)
    connect4 = Connect4()
    while True:
        connect4.printState()
        while True: 
            try:
                move = int(input("Choose your move: "))
                connect4.updateGameState(move)
                break
            except ValueError as e:
                print(f"{e}")
                continue 

        mcts.updateAfterAdversaryTurn(connect4, move)
        
        if connect4.checkPlayerWon(player="O"):
            connect4.printState()
            print("You win")
            break
        elif connect4.checkTie():
            print("Tie")
            break

        connect4.printState()
        print("AI is thinking...")
        mcts.run_mcts(iterations, connect4)
        bestMove = mcts.choose_best_move(showNodesStats)
        connect4.updateGameState(bestMove)

        if connect4.checkPlayerWon(player="X"):
            connect4.printState()
            print("AI win")
            break
        elif connect4.checkTie():
            print("Tie")
            break

    if showMCTSTime:
        print(f"MCTS: {mcts.runTimes}")

    del mcts
    del connect4
    gc.collect()

def AI_first_vs_user(c_constant_mcts: float, iterations: int, reset: bool, drawValue: float, showMCTSTime: bool, showNodesStats: bool):
    mcts = MctsAlgo(C=c_constant_mcts, reset=reset, drawValue=drawValue)
    connect4 = Connect4()
    while True:
        connect4.printState()
        print("AI is thinking...")
        mcts.run_mcts(iterations, connect4)
        bestMove = mcts.choose_best_move(showNodesStats)
        connect4.updateGameState(bestMove)

        if connect4.checkPlayerWon(player="O"):
            connect4.printState()
            print("AI win")
            break
        elif connect4.checkTie():
            print("Tie")
            break

        connect4.printState()
        while True: 
            try:
                move = int(input("Choose your move: "))
                connect4.updateGameState(move)
                break
            except ValueError as e:
                print(f"{e}")
                continue 
        
        mcts.updateAfterAdversaryTurn(connect4, move)

        if connect4.checkPlayerWon(player="X"):
            connect4.printState()
            print("You win")
            break
        elif connect4.checkTie():
            print("Tie")
            break

    if showMCTSTime:
        print(f"MCTS: {mcts.runTimes}")
        
    del mcts
    del connect4
    gc.collect()

def AI_vs_AI(c_constant_mcts_1st: float, iterations_1st: int, reset1: bool, drawValue1: float, c_constant_mcts_2nd: float, iterations_2nd: int, reset2: bool, drawValue2: float, showMCTSTime: bool, showNodesStats: bool):
    connect4 = Connect4()
    mcts1 = MctsAlgo(C=c_constant_mcts_1st, reset=reset1, drawValue=drawValue1)
    mcts2 = MctsAlgo(C=c_constant_mcts_2nd, reset=reset2, drawValue=drawValue2)    
    while True:
        connect4.printState()
        print("AI_1st is thinking...")
        mcts1.run_mcts(iterations_1st, connect4)
        bestMove = mcts1.choose_best_move(showNodesStats)
        connect4.updateGameState(bestMove)
        mcts2.updateAfterAdversaryTurn(connect4, bestMove)

        if connect4.checkPlayerWon(player="O"):
            connect4.printState()
            print("AI_1st win")
            break
        elif connect4.checkTie():
            connect4.printState()
            print("Tie")
            break

        connect4.printState()
        print("AI_2nd is thinking...")
        mcts2.run_mcts(iterations_2nd, connect4)
        bestMove = mcts2.choose_best_move(showNodesStats)
        connect4.updateGameState(bestMove)
        mcts1.updateAfterAdversaryTurn(connect4, bestMove)

        if connect4.checkPlayerWon(player="X"):
            connect4.printState()
            print("AI_2nd win")
            break
        elif connect4.checkTie():
            connect4.printState()
            print("Tie")
            break
    
    if showMCTSTime:
        print(f"MCTS1: {mcts1.runTimes}")
        print(f"MCTS2: {mcts2.runTimes}")

    mcts1.currentState.children = {}   
    mcts1.actualState.children = {}   
    mcts2.currentState.children = {} 
    mcts2.actualState.children = {}  
    del mcts1
    del mcts2
    del connect4
    gc.collect()

def Benchmarking_AI_vs_AI(c_constant_mcts_1st: float, iterations_1st: int, reset1: bool, drawValue1: float, c_constant_mcts_2nd: float, iterations_2nd: int, reset2: bool, drawValue2: float, showMCTSTime: bool, showNodesStats: bool, dataset):
    connect4 = Connect4()
    mcts1 = MctsAlgo(C=c_constant_mcts_1st, reset=reset1, drawValue=drawValue1)
    mcts2 = MctsAlgo(C=c_constant_mcts_2nd, reset=reset2, drawValue=drawValue2)
    while True:
        mcts1.run_mcts(iterations_1st, connect4, dataset)
        bestMove = mcts1.choose_best_move(showNodesStats)
        connect4.updateGameState(bestMove)
        mcts2.updateAfterAdversaryTurn(connect4, bestMove)

        if connect4.checkPlayerWon(player="O"):
            break
        elif connect4.checkTie():
            break

        mcts2.run_mcts(iterations_2nd, connect4, dataset)
        bestMove = mcts2.choose_best_move(showNodesStats)
        connect4.updateGameState(bestMove)
        mcts1.updateAfterAdversaryTurn(connect4, bestMove)

        if connect4.checkPlayerWon(player="X"):
            break
        elif connect4.checkTie():
            break
    
    if showMCTSTime:
        print(f"MCTS1: {mcts1.runTimes}")
        print(f"MCTS2: {mcts2.runTimes}")

    mcts1.currentState.children = {}   
    mcts1.actualState.children = {}   
    mcts2.currentState.children = {} 
    mcts2.actualState.children = {}  
    del mcts1
    del mcts2
    del connect4
    gc.collect()

def read_input(config_filePath: str):
    resets = []
    Cs = []
    with open(config_filePath, "r") as file:
        lines = file.readlines()
        C0 = lines[1].strip().split(" = ")[1]
        Cs.append(C0)
        iterations0 = int(lines[2].strip().split(" = ")[1])
        resetTree0 = lines[3].strip().split(" = ")[1]
        resets.append(resetTree0)
        drawValue0 = float(lines[4].strip().split(" = ")[1])
        C1 = lines[7].strip().split(" = ")[1]
        Cs.append(C1)
        iterations1 = int(lines[8].strip().split(" = ")[1])
        resetTree1 = lines[9].strip().split(" = ")[1]
        resets.append(resetTree1)
        drawValue1 = float(lines[10].strip().split(" = ")[1])
        C2 = lines[11].strip().split(" = ")[1]
        Cs.append(C2)
        iterations2 = int(lines[12].strip().split(" = ")[1])
        resetTree2 = lines[13].strip().split(" = ")[1]
        resets.append(resetTree2)
        drawValue2 = float(lines[14].strip().split(" = ")[1])
        showMCTSTime = lines[17].strip().split(" = ")[1]
        showNodesStats = lines[20].strip().split(" = ")[1]
        benchmarkingFile = lines[23].strip().split(" = ")[1]

    for i in range(len(Cs)):
        if Cs[i][0:5] == "sqrt_":
            Cs[i] = math.sqrt(int(Cs[i][5]))
        else:
            Cs[i] = float(Cs[i])

    for i in range(len(resets)):
        if resets[i].lower() == "true":
            resets[i] = True
        else:
            resets[i] = False

    if showMCTSTime.lower() == "true":
        showMCTSTime = True
    else:
        showMCTSTime = False

    if showNodesStats.lower() == "true":
        showNodesStats = True
    else:
        showNodesStats = False

    return {"C0": Cs[0], "iterations0": iterations0, "resetTree0": resets[0], "drawValue0": drawValue0,
            "C1": Cs[1], "iterations1": iterations1, "resetTree1": resets[1], "drawValue1": drawValue1, 
            "C2": Cs[2], "iterations2": iterations2, "resetTree2": resets[2], "drawValue2": drawValue2,
            "showMCTSTime": showMCTSTime, "showNodesStats": showNodesStats, "benchmarkingFile": benchmarkingFile}

def save_to_csv(dataset: list[list[str]], filePath: str):
    col_names = [f"cell_{i}" for i in range(42)] + ["10kIter", "20kIter", "30kIter", "40kIter", "50kIter",]
    df_new = pd.DataFrame(dataset, columns=col_names)
    csv_path = f'other_version/datasets/{filePath}'

    if os.path.exists(csv_path):
        df_existing = pd.read_csv(csv_path)
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        df_combined.to_csv(csv_path, index=False)
        print(f"\nAppended to existing connect4_{filePath}.csv (total: {len(df_combined)} rows)")
    else:
        df_new.to_csv(csv_path, index=False)
        print(f"\nDataset saved as new connect4_{filePath}.csv")

if __name__ == "__main__":
    typeOfGame = ""
    config = read_input(r"..\..\configs\configs.txt")
    showMCTSTime = config["showMCTSTime"]
    showNodesStats = config["showNodesStats"]
    while typeOfGame.lower() != "exit":
        print("""
<---------------------------------------->
        What do you want to play:

            1. User vs AI
            2. AI vs User
             3. AI vs AI
         4. Benchmarking AIs

              -- Exit --
<---------------------------------------->
""")
        typeOfGame = input("Choose: ")
        if typeOfGame == "1":
            C_constant, nIterations, reset, drawValue = config["C0"], config["iterations0"], config["resetTree0"], config["drawValue0"]
            user_first_vs_AI(C_constant, nIterations, reset, drawValue, showMCTSTime, showNodesStats)

        elif typeOfGame == "2":
            C_constant, nIterations, reset, drawValue = config["C0"], config["iterations0"], config["resetTree0"], config["drawValue0"]
            AI_first_vs_user(C_constant, nIterations, reset, drawValue, showMCTSTime, showNodesStats)

        elif typeOfGame == "3":
            C_constant1, nIterations1, reset1, drawValue1 = config["C1"], config["iterations1"], config["resetTree1"], config["drawValue1"]
            C_constant2, nIterations2, reset2, drawValue2 = config["C2"], config["iterations2"], config["resetTree2"], config["drawValue2"]

            AI_vs_AI(C_constant1, nIterations1, reset1, drawValue1, C_constant2, nIterations2, reset2, drawValue2, showMCTSTime, showNodesStats)


        elif typeOfGame == "4":
            # Concurrent AI vs AI
            C_constant1, nIterations1, reset1, drawValue1 = config["C1"], config["iterations1"], config["resetTree1"], config["drawValue1"]
            C_constant2, nIterations2, reset2, drawValue2 = config["C2"], config["iterations2"], config["resetTree2"], config["drawValue2"]
            
            manager = Manager()
            dataset = manager.list()
            total_iterations = 200

            with tqdm(total=total_iterations, desc="Benchmarking Progress", unit="iteration") as pbar:
                for i in range(total_iterations):
                    # Create and start 5 processes
                    processes = []
                    for _ in range(5):
                        p = Process(target=Benchmarking_AI_vs_AI, args=(C_constant1, nIterations1, reset1, drawValue1, C_constant2, nIterations2, reset2, drawValue2, showMCTSTime, showNodesStats, dataset))
                        processes.append(p)
                        p.start()
                    # Wait for all processes to complete
                    for p in processes:
                        p.join()
                    # Update the progress bar
                    pbar.update(1)
            
            dataset = list(dataset)
            # Write results to csv file
            save_to_csv(dataset=dataset, filePath = config["benchmarkingFile"])

        else:
            if typeOfGame.lower() != "exit":
                print("\nPlease choose between the options available!\n")


<---------------------------------------->
        What do you want to play:

            1. User vs AI
            2. AI vs User
             3. AI vs AI
         4. Benchmarking AIs

              -- Exit --
<---------------------------------------->



Benchmarking Progress: 100%|██████████| 1000/1000 [00:00<00:00, 2177.04game/s]


OSError: Cannot save file into a non-existent directory: 'other_version\datasets'